In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import time
from IPython import display

from dedalus import public as de
from dedalus.extras import flow_tools

import logging
logger = logging.getLogger(__name__)

In [ ]:
Lx, Ly = (40, 40)
nx, ny = (300, 300)

# Create bases and domain
x_basis = de.Fourier('x', nx, interval=(0, Lx), dealias=3/2)
y_basis = de.Fourier('y', ny, interval=(0, Ly), dealias=3/2)
domain = de.Domain([x_basis, y_basis], grid_dtype=np.float64)

In [ ]:
def grow_fun(A,B):
    F = (A.data)*(1-B.data)
    F[A.data < 1e-3] = 0
    return F


def grow_operator(field1,field2):
    return de.operators.GeneralFunction(
        field1.domain,
        layout = 'g',
        func = grow_fun,
        args = (field1,field2,)
    )

de.operators.parseables['grow'] = grow_operator

In [ ]:
params = np.load('Parameters.npz')
Diff = params['Diff'].flat[0]
K =  params['K'].flat[0]
g = params['g'].flat[0]

problem = de.IVP(domain, variables=['rho1','rho2','rho12'])
problem.parameters['D0'] = Diff
problem.parameters['K'] = K
problem.parameters['g1'] = g
problem.parameters['g2'] = g

problem.substitutions['rho_b'] = "rho1+rho2+rho12" 
problem.substitutions["Lap(A)"] = "dx(dx(A)) + dy(dy(A))"


problem.add_equation("dt(rho1)  - D0*Lap(rho1) = g1*grow(rho1,rho_b)  - K*rho1*rho2")
problem.add_equation("dt(rho2)  - D0*Lap(rho2) = g2*grow(rho2,rho_b)  - K*rho1*rho2")
problem.add_equation("dt(rho12)  = 2*K*rho1*rho2")



In [ ]:
ts = de.timesteppers.RK443
solver =  problem.build_solver(ts)

In [ ]:
x = domain.grid(0)
y = domain.grid(1)
rho1 = solver.state['rho1']
rho2 = solver.state['rho2']

def r(x,y,x0,y0):
    return np.sqrt((x-x0)**2+(y-y0)**2)

def blob(x,y,x0,y0,r0,w):
    return 0.5*(1 - np.tanh((r(x,y,x0,y0)-r0)/w))

def r_p(y,y0):
    return np.abs(y-y0)
def circle(x,y,y0,r0,w):
    return 0.5*(1 - np.tanh((r_p(y,y0)-r0)/w))


r0 = 0.6
rl = 0.3
w = 0.15
wl = 0.15

rho1['g'] = blob(x,y,Lx*0.5,Ly*0.5,r0,w)
rho2['g'] = blob(x,y,Lx*0.5,Ly*0.5,18.6,w) - blob(x,y,Lx*0.5,Ly*0.5,18,w)

In [ ]:
solver.stop_sim_time = 250
dt = 0.3*Lx/nx

In [ ]:
analysis = solver.evaluator.add_file_handler('circle', sim_dt=2, max_writes=500)
analysis.add_task('rho1')
analysis.add_task('rho2')
analysis.add_task('rho12')

In [ ]:
logger.info('Starting loop')
start_time = time.time()
while solver.ok:
    solver.step(dt)
    if solver.iteration % 50 == 0:
        logger.info('Iteration: %i, Time: %e, dt: %e' %(solver.iteration, solver.sim_time, dt))

In [ ]:
from dedalus.tools import post
post.merge_process_files("circle", cleanup=True)